In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis/Output')
cur_date = "050423"

library(readxl)
library(tidyverse)
library(reshape2)
library(stats)
library(multcomp)

# reading in files
cytokine_df = data.frame(read_excel("Input/Cytokine_Data_050423.xlsx", sheet = 2))
demographics_df = data.frame(read_excel("Input/Subject_Info_031723.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select



Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser


Warning message:
“Expecting numeric in E1226 / R1226C5: got 'NA'”
Warning message:
“Expecting numeric in E1227 / R1227C5: got 'NA'”
Warning message:
“Expecting numeric in E1228 

Testing for statistical differences by comparing a cytokine concentration between burn conditions (control, smoldering or flaming). (The condensate and concentration would remain consistent). Cytokine concentration was pseudo log transformed to reduce skew.

This time sex will be added to models as a covariate. I couldn't find a non-parametric test in R that would allow for the addition of a covariate, therefore crude ANOVA models/ Tukey's post hoc test will be run first and then ANCOVA models/ Tukey's post hoc test that include sex.

In [2]:
head(cytokine_df)
head(demographics_df)

,Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,1,F_1,C,F,1,Eotaxin3,21.750958,4.507855
2,1,F_1,C,F,25,Eotaxin3,11.624224,3.658123
3,1,F_1,C,F,5,Eotaxin3,17.828714,4.234863
4,1,F_1,C,S,1,Eotaxin3,22.798534,4.572801
5,1,F_1,C,S,25,Eotaxin3,10.342567,3.503675
6,1,F_1,C,S,5,Eotaxin3,9.140652,3.342078


,Original_Subject_ID,Subject_ID,Subject_No,Sex,Age,Race,Ethnicity
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,68J,F_1,1,F,18,B,NH
2,53K,M_2,2,M,32,B,NH
3,21O,M_3,3,M,34,W,H
4,63O,F_4,4,F,32,W,H
5,67L,F_5,5,F,17,W,H
6,57N,M_6,6,M,59,W,NH


In [3]:
# combining data, but first making sex a factor
demographics_df = demographics_df %>%
    mutate(Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0"))

cytokine_sex_df = inner_join(cytokine_df, demographics_df[,c(2,4)]) %>%
    # scaling the concentration values by protein normalizes the distribution
    group_by(Cytokine) %>%
    mutate(Scaled_Conc_pslog2 = scale(Cytokine_Conc_pslog2))

# putting burn condition into a factor
cytokine_sex_df$Burn_Condition = factor(cytokine_sex_df$Burn_Condition, levels = c("PBS", "S", "F"))

head(cytokine_sex_df)

Joining, by = "Subject_ID"


Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2,Sex,Scaled_Conc_pslog2
<dbl>,<chr>,<chr>,<fct>,<dbl>,<chr>,<dbl>,<dbl>,<fct>,"<dbl[,1]>"
1,F_1,C,F,1,Eotaxin3,21.750958,4.507855,0,1.2619321
1,F_1,C,F,25,Eotaxin3,11.624224,3.658123,0,0.4831684
1,F_1,C,F,5,Eotaxin3,17.828714,4.234863,0,1.0117394
1,F_1,C,S,1,Eotaxin3,22.798534,4.572801,0,1.3214533
1,F_1,C,S,25,Eotaxin3,10.342567,3.503675,0,0.3416201
1,F_1,C,S,5,Eotaxin3,9.140652,3.342078,0,0.1935198


In [4]:
# contrasts show what compartments are being compared in anova
# these comparisons aren't what we want so I changed them in the function below
contrasts(cytokine_sex_df$Burn_Condition)

,S,F
PBS,0,0
S,1,0
F,0,1


               Table of Contrasts 
>               PBS |  S  |  F  |  Sum
>  - Contrast 1 |  -1  |  1  |  0  |   0
>  - Contrast 2 |  -1  |  0  |  1  |   0

> - Contrast 1: compares PBS to smoldering
> - Contrast 2: compares PBS to flaming

In [5]:
cytokine_df %>%
    filter(Cytokine == "IL10", Condensate %in% c("C", "PBS"), Condensate_Conc %in% c(1, NA))

Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,F_1,C,F,1,IL10,0.38855032,0.47357946
1,F_1,C,S,1,IL10,0.05697711,0.07994413
2,M_2,C,F,1,IL10,0.23498190,0.30448990
2,M_2,C,S,1,IL10,0.35465972,0.43793050
3,M_3,C,F,1,IL10,0.31639620,0.39659376
3,M_3,C,S,1,IL10,0.31407751,0.39405038
4,F_4,C,F,1,IL10,1.33211085,1.22163637
4,F_4,C,S,1,IL10,1.98867598,1.57950650
5,F_5,C,F,1,IL10,1.60022893,1.37863865


In [6]:
get_anova = function(df){
    # """
    # Running anova/ tukey's tests after filtering for a cytokine and condensate using a loop. 
    # Ultimately using this test to compare cytokine expression (control vs. flaming burn condition).

    # :param: cytokine dataframe, concentration
    # :output: a dataframe containing the cytokine, condensate, comparison, stat, p value, p adj

    # """
    
    # first filtering the df and iterating through the variables
    cytokines = unique(df$Cytokine)
    condensates = c("C", "P")
    conc = unique(df$Condensate_Conc)[1:3]
    
    tukey_df = data.frame()
    for(i in 1:length(cytokines)){
        for(j in 1:length(condensates)){
            for(k in 1:length(conc)){
                filtered_df = df %>%
                    filter(Cytokine == cytokines[i], Condensate %in% c(condensates[j], "PBS"), 
                           Condensate_Conc %in% c(conc[k], NA))

                # changing contrasts
                contrasts(filtered_df$Burn_Condition) = cbind(c(-1,1,0), c(-1,0,1)) # meaning is specified above
                # ANOVA
                anova = aov(Scaled_Conc_pslog2 ~ Burn_Condition, data = filtered_df)

                # Tukey's post hoc test
                tukeys_anova = summary(glht(anova, linfct = mcp(Burn_Condition = "Tukey")), 
                                       test = adjusted("none"))
                tukeys_test = tukeys_anova$test
                values_df = data.frame("ANOVA", condensates[j], cytokines[i], conc[k], tukeys_test$tstat, 
                                       tukeys_test$pvalues) %>%
                    rownames_to_column(var = "Comparison")

                # saving values
                tukey_df = rbind(tukey_df, values_df)
            }
        }
    }
    
    # changing col names
    colnames(tukey_df)[2:7] = c("Model", "Condensate", "Cytokine", "Concentration", "Statistic", 
                                "P Value")
    
 
    # calculating padj values
    PAdj = c()
    for(j in 1:length(condensates)){
        filtered_df = tukey_df %>%
            filter(Condensate == condensates[j])
        padj = p.adjust(as.numeric(as.character(filtered_df$`P Value`)), method = "fdr")
        PAdj = c(PAdj, padj)

    }
    
   tukey_df$`P Adj` = PAdj
    
    return(tukey_df)
}

# calling fn
cytokine_anova_tukey = get_anova(cytokine_sex_df)

In [7]:
get_ancova = function(df){
    # """
    # Running ancova/ tukey's tests after filtering for a cytokine and condensate using a loop. 
    # Ultimately using this test to compare cytokine expression (control vs. flaming burn condition).

    # :param: cytokine dataframe, concentration
    # :output: a dataframe containing the cytokine, condensate, comparison, stat, p value, p adj

    # """
    
    # first filtering the df and iterating through the variables
    cytokines = unique(df$Cytokine)
    condensates = c("C", "P")
    conc = unique(df$Condensate_Conc)[1:3]
    
    tukey_df = data.frame()
    for(i in 1:length(cytokines)){
        for(j in 1:length(condensates)){
            for(k in 1:length(conc)){
                filtered_df = df %>%
                    filter(Cytokine == cytokines[i], Condensate %in% c(condensates[j], "PBS"), 
                           Condensate_Conc %in% c(conc[k], NA))

                # changing contrasts
                contrasts(filtered_df$Burn_Condition) = cbind(c(-1,1,0), c(-1,0,1)) # meaning is specified above
                # ANCOVA
                ancova = aov(Scaled_Conc_pslog2 ~ Burn_Condition, data = filtered_df)

                # Tukey's post hoc test
                tukeys_ancova = summary(glht(ancova, linfct = mcp(Burn_Condition = "Tukey")), 
                                       test = adjusted("none"))
                tukeys_test = tukeys_ancova$test
                values_df = data.frame("ANCOVA", condensates[j], cytokines[i], conc[k], tukeys_test$tstat, 
                                       tukeys_test$pvalues) %>%
                    rownames_to_column(var = "Comparison")

                # saving values
                tukey_df = rbind(tukey_df, values_df)
            }
        }
    }
    
    # changing col names
    colnames(tukey_df)[2:7] = c("Model", "Condensate", "Cytokine", "Concentration", "Statistic", 
                                "P Value")
    
 
    # calculating padj values
    PAdj = c()
    for(j in 1:length(condensates)){
        filtered_df = tukey_df %>%
            filter(Condensate == condensates[j])
        padj = p.adjust(as.numeric(as.character(filtered_df$`P Value`)), method = "fdr")
        PAdj = c(PAdj, padj)

    }
    
   tukey_df$`P Adj` = PAdj
    
    return(tukey_df)
}

# calling fn
cytokine_ancova_tukey = get_ancova(cytokine_sex_df)

In [8]:
# creating 1 df 
anova_ancova_df = rbind(cytokine_anova_tukey, cytokine_ancova_tukey) %>%
    # changing condensates so they're more legible
    mutate(Condensate = ifelse(Condensate == "C", "Cardboard",
                               ifelse(Condensate == "P", "Plastic", "PBS")))
head(anova_ancova_df)

,Comparison,Model,Condensate,Cytokine,Concentration,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,S - PBS,ANOVA,Cardboard,Eotaxin3,1,1.3794751,0.1879682,0.8216894
2,F - PBS,ANOVA,Cardboard,Eotaxin3,1,1.0073952,0.3297201,0.9008425
3,F - S,ANOVA,Cardboard,Eotaxin3,1,-0.3720799,0.7150374,0.9953057
4,S - PBS,ANOVA,Cardboard,Eotaxin3,25,0.1049463,0.9178089,0.9953057
5,F - PBS,ANOVA,Cardboard,Eotaxin3,25,-0.4490638,0.6598056,0.9953057
6,F - S,ANOVA,Cardboard,Eotaxin3,25,-0.5540100,0.5877367,0.9953057


In [9]:
# exporting
write.csv(anova_ancova_df, paste0(Output,"/", cur_date, "_Cytokine_ANOVA_ANCOVA.csv"), row.names = FALSE)